In [43]:
from pathlib import Path
from fishsense_data_processing_workflow_worker.config import settings
from fishsense_api_sdk.client import Client
from label_studio_sdk.client import LabelStudio
from datetime import datetime
from fishsense_api_sdk.models.laser_label import LaserLabel
from tqdm.notebook import tqdm
from copy import deepcopy
import cv2
import asyncio

In [12]:
DIVE_ID = 393
PROJECT_ID = 72

In [13]:
OUTPUT_FOLDER = (Path("../output") / "preprocess_jpeg").absolute()

OUTPUT_FOLDER.exists()

True

In [14]:
label_studio_api_key = Path("..") / ".label_studio_api_key"

label_studio_api_key.exists()

True

In [15]:
ls = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key=label_studio_api_key.read_text().strip())

In [16]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as fs:
    dive = await fs.dives.get(dive_id=DIVE_ID)

dive

Dive(id=393, name='062824_Alligator1_FSL01', path='2025-02-10 REEF Data Dump SMILE 6/062824_Alligator_FSL01/062824_Alligator1_FSL01', dive_datetime=datetime.datetime(2024, 6, 27, 22, 51, 58, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=1, dive_slate_id=None)

In [17]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as fs:
    images = await fs.images.get(dive_id=dive.id)

len(images), images[0]

(241,
 Image(id=113697, path='2025-02-10 REEF Data Dump SMILE 6/062824_Alligator_FSL01/062824_Alligator1_FSL01/P6270091.ORF', taken_datetime=datetime.datetime(2024, 6, 27, 21, 58, 51, tzinfo=TzInfo(0)), checksum='3c2e38a56aca2535ce2471b089e8bfcc', is_canonical=True, dive_id=393, camera_id=1))

In [29]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as client:
    existing_laser_labels = await client.labels.get_laser_labels(dive.id)

len(existing_laser_labels), existing_laser_labels

(30,
 [LaserLabel(id=6602, label_studio_task_id=191611, label_studio_project_id=43, x=1925.656571218975, y=1323.5406071053478, label='Green Laser', updated_at=datetime.datetime(2025, 4, 22, 20, 35, 37, 191313, tzinfo=TzInfo(0)), superseded=False, completed=True, label_studio_json={'annotations': [{'id': 85200, 'result': [{'id': 'Luruui7XoE', 'type': 'keypointlabels', 'value': {'x': 47.973507005953536, 'y': 43.883972384129564, 'width': 0.30236225524547383, 'keypointlabels': ['Green Laser']}, 'origin': 'manual', 'to_name': 'img-1', 'from_name': 'kp-1', 'image_rotation': 0, 'original_width': 4014, 'original_height': 3016}], 'created_username': ' jessarmacsot@gmail.com, 46', 'created_ago': '8\xa0months, 1\xa0week', 'completed_by': 46, 'was_cancelled': False, 'ground_truth': False, 'created_at': '2025-04-22T20:35:37.157669Z', 'updated_at': '2025-04-22T20:35:37.157680Z', 'draft_created_at': None, 'lead_time': 18.641, 'import_id': None, 'last_action': None, 'bulk_created': False, 'task': 1916

In [30]:
laser_labels_by_image_id = {label.image_id: label for label in existing_laser_labels}

len(laser_labels_by_image_id), laser_labels_by_image_id

(30,
 {113779: LaserLabel(id=6602, label_studio_task_id=191611, label_studio_project_id=43, x=1925.656571218975, y=1323.5406071053478, label='Green Laser', updated_at=datetime.datetime(2025, 4, 22, 20, 35, 37, 191313, tzinfo=TzInfo(0)), superseded=False, completed=True, label_studio_json={'annotations': [{'id': 85200, 'result': [{'id': 'Luruui7XoE', 'type': 'keypointlabels', 'value': {'x': 47.973507005953536, 'y': 43.883972384129564, 'width': 0.30236225524547383, 'keypointlabels': ['Green Laser']}, 'origin': 'manual', 'to_name': 'img-1', 'from_name': 'kp-1', 'image_rotation': 0, 'original_width': 4014, 'original_height': 3016}], 'created_username': ' jessarmacsot@gmail.com, 46', 'created_ago': '8\xa0months, 1\xa0week', 'completed_by': 46, 'was_cancelled': False, 'ground_truth': False, 'created_at': '2025-04-22T20:35:37.157669Z', 'updated_at': '2025-04-22T20:35:37.157680Z', 'draft_created_at': None, 'lead_time': 18.641, 'import_id': None, 'last_action': None, 'bulk_created': False, 'tas

In [31]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as client:
    dive_images = await client.images.get(dive_id=DIVE_ID)

len(dive_images), dive_images

(241,
 [Image(id=113697, path='2025-02-10 REEF Data Dump SMILE 6/062824_Alligator_FSL01/062824_Alligator1_FSL01/P6270091.ORF', taken_datetime=datetime.datetime(2024, 6, 27, 21, 58, 51, tzinfo=TzInfo(0)), checksum='3c2e38a56aca2535ce2471b089e8bfcc', is_canonical=True, dive_id=393, camera_id=1),
  Image(id=113698, path='2025-02-10 REEF Data Dump SMILE 6/062824_Alligator_FSL01/062824_Alligator1_FSL01/P6270094.ORF', taken_datetime=datetime.datetime(2024, 6, 27, 21, 58, 52, tzinfo=TzInfo(0)), checksum='9f1801d273ecc9c4b1746a58b946252d', is_canonical=True, dive_id=393, camera_id=1),
  Image(id=113699, path='2025-02-10 REEF Data Dump SMILE 6/062824_Alligator_FSL01/062824_Alligator1_FSL01/P6270093.ORF', taken_datetime=datetime.datetime(2024, 6, 27, 21, 58, 52, tzinfo=TzInfo(0)), checksum='4c9e18af6f7d815976f476fe240a277d', is_canonical=True, dive_id=393, camera_id=1),
  Image(id=113700, path='2025-02-10 REEF Data Dump SMILE 6/062824_Alligator_FSL01/062824_Alligator1_FSL01/P6270092.ORF', taken_

In [32]:
incomplete_images = [image for image in dive_images if laser_labels_by_image_id.get(image.id) is None or not laser_labels_by_image_id[image.id].completed]

len(incomplete_images), incomplete_images

(211,
 [Image(id=113698, path='2025-02-10 REEF Data Dump SMILE 6/062824_Alligator_FSL01/062824_Alligator1_FSL01/P6270094.ORF', taken_datetime=datetime.datetime(2024, 6, 27, 21, 58, 52, tzinfo=TzInfo(0)), checksum='9f1801d273ecc9c4b1746a58b946252d', is_canonical=True, dive_id=393, camera_id=1),
  Image(id=113700, path='2025-02-10 REEF Data Dump SMILE 6/062824_Alligator_FSL01/062824_Alligator1_FSL01/P6270092.ORF', taken_datetime=datetime.datetime(2024, 6, 27, 21, 58, 52, tzinfo=TzInfo(0)), checksum='e29a3ef90e913653dd2acc1e3ec410f5', is_canonical=True, dive_id=393, camera_id=1),
  Image(id=113701, path='2025-02-10 REEF Data Dump SMILE 6/062824_Alligator_FSL01/062824_Alligator1_FSL01/P6270095.ORF', taken_datetime=datetime.datetime(2024, 6, 27, 21, 58, 58, tzinfo=TzInfo(0)), checksum='a619b9bd3d1a776b3808737f50b2f7e0', is_canonical=True, dive_id=393, camera_id=1),
  Image(id=113702, path='2025-02-10 REEF Data Dump SMILE 6/062824_Alligator_FSL01/062824_Alligator1_FSL01/P6270096.ORF', taken_

In [19]:
now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

now

'2026-01-03 22:33:12'

In [33]:
tasks = []
template = {
    "model_version": f"sql_labels_{now}",
    "result": []
}

for image in tqdm(incomplete_images):
    image_path = OUTPUT_FOLDER / f"{image.checksum}.JPG"

    img = cv2.imread(str(image_path))
    height, width, _ = img.shape

    tasks.append({
        "data": {
            "image": f"https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/{image.checksum}"
        },
        "annotations": []
    })

  0%|          | 0/211 [00:00<?, ?it/s]

In [34]:
tasks

[{'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/9f1801d273ecc9c4b1746a58b946252d'},
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/e29a3ef90e913653dd2acc1e3ec410f5'},
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/a619b9bd3d1a776b3808737f50b2f7e0'},
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/4253b4bbe7825eb15e9e5659acdb2409'},
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/51d023ba8fe11f1fe5990cf10b85a202'},
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/058c5d3bf10209a18ff62ac0961b5b7c'},
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/e

In [38]:
imported_tasks = ls.projects.import_tasks(PROJECT_ID, request=tasks, return_task_ids=True)

imported_tasks

ImportTasksProjectsResponse(annotation_count=0, could_be_tasks_list=False, data_columns=[], duration=1.1006929874420166, file_upload_ids=[], found_formats=[], predictions_count=None, task_count=211, prediction_count=0, task_ids=[224793, 224794, 224795, 224796, 224797, 224798, 224799, 224800, 224801, 224802, 224803, 224804, 224805, 224806, 224807, 224808, 224809, 224810, 224811, 224812, 224813, 224814, 224815, 224816, 224817, 224818, 224819, 224820, 224821, 224822, 224823, 224824, 224825, 224826, 224827, 224828, 224829, 224830, 224831, 224832, 224833, 224834, 224835, 224836, 224837, 224838, 224839, 224840, 224841, 224842, 224843, 224844, 224845, 224846, 224847, 224848, 224849, 224850, 224851, 224852, 224853, 224854, 224855, 224856, 224857, 224858, 224859, 224860, 224861, 224862, 224863, 224864, 224865, 224866, 224867, 224868, 224869, 224870, 224871, 224872, 224873, 224874, 224875, 224876, 224877, 224878, 224879, 224880, 224881, 224882, 224883, 224884, 224885, 224886, 224887, 224888, 224

In [42]:
laser_labels = [LaserLabel(id=None, image_id=image.id, label_studio_task_id=task_id, label_studio_project_id=PROJECT_ID, image_url=f"https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/{image.checksum}", updated_at=None, completed=False, label_studio_json={}, user_id=None, superseded=False, x=None, y=None, label=None) for task_id, image in zip(imported_tasks.task_ids, incomplete_images)]

len(laser_labels), laser_labels

(211,
 [LaserLabel(id=None, label_studio_task_id=224793, label_studio_project_id=72, x=None, y=None, label=None, updated_at=None, superseded=False, completed=False, label_studio_json={}, image_id=113698, user_id=None),
  LaserLabel(id=None, label_studio_task_id=224794, label_studio_project_id=72, x=None, y=None, label=None, updated_at=None, superseded=False, completed=False, label_studio_json={}, image_id=113700, user_id=None),
  LaserLabel(id=None, label_studio_task_id=224795, label_studio_project_id=72, x=None, y=None, label=None, updated_at=None, superseded=False, completed=False, label_studio_json={}, image_id=113701, user_id=None),
  LaserLabel(id=None, label_studio_task_id=224796, label_studio_project_id=72, x=None, y=None, label=None, updated_at=None, superseded=False, completed=False, label_studio_json={}, image_id=113702, user_id=None),
  LaserLabel(id=None, label_studio_task_id=224797, label_studio_project_id=72, x=None, y=None, label=None, updated_at=None, superseded=False, 

In [44]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as fs:
    async with asyncio.TaskGroup() as tg:
        for laser_label in tqdm(laser_labels):
            tg.create_task(fs.labels.put_laser_label(laser_label.image_id, laser_label))

  0%|          | 0/211 [00:00<?, ?it/s]